In [1]:
import duckdb

In [2]:
con = duckdb.connect()
con.execute("INSTALL httpfs; LOAD httpfs;")
con.execute("INSTALL ducklake; LOAD ducklake;")

In [3]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'marketing-and-ecommerce',
    'endpoint': 'localhost:9000'
}

con.sql(f"SET s3_endpoint = '{minioAuth['endpoint']}';")
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_use_ssl = false;") # Wajib untuk http (tanpa ssl)
con.sql("SET s3_url_style = 'path';") # Wajib untuk MinIO

In [4]:
catalogMetadata = '../dataLakehouse/catalog.ducklake'

In [5]:
con.sql(f"ATTACH 'ducklake:{catalogMetadata}' AS dataLakehouse;")
con.sql("USE dataLakehouse;")

In [6]:
con.sql("SELECT * FROM ducklake_snapshots('dataLakehouse')")

┌─────────────┬───────────────────────────────┬────────────────┬───────────────────────────────────────────────────────────────┬─────────┬────────────────┬───────────────────┐
│ snapshot_id │         snapshot_time         │ schema_version │                            changes                            │ author  │ commit_message │ commit_extra_info │
│    int64    │   timestamp with time zone    │     int64      │                    map(varchar, varchar[])                    │ varchar │    varchar     │      varchar      │
├─────────────┼───────────────────────────────┼────────────────┼───────────────────────────────────────────────────────────────┼─────────┼────────────────┼───────────────────┤
│           0 │ 2025-12-12 14:41:22.425269+07 │              0 │ {schemas_created=[main]}                                      │ NULL    │ NULL           │ NULL              │
│           1 │ 2025-12-12 14:41:22.580958+07 │              1 │ {schemas_created=[raw]}                                

In [7]:
con.sql("SELECT * FROM information_schema.schemata;")

┌───────────────────────────────────┬──────────────────────────┬──────────────┬───────────────────────────────┬──────────────────────────────┬────────────────────────────┬──────────┐
│           catalog_name            │       schema_name        │ schema_owner │ default_character_set_catalog │ default_character_set_schema │ default_character_set_name │ sql_path │
│              varchar              │         varchar          │   varchar    │            varchar            │           varchar            │          varchar           │ varchar  │
├───────────────────────────────────┼──────────────────────────┼──────────────┼───────────────────────────────┼──────────────────────────────┼────────────────────────────┼──────────┤
│ __ducklake_metadata_dataLakehouse │ main                     │ duckdb       │ NULL                          │ NULL                         │ NULL                       │ NULL     │
│ __ducklake_metadata_dataLakehouse │ sqlmesh                  │ duckdb       │ NULL 

In [8]:
con.sql("SHOW TABLES FROM dataLakehouse;")

┌──────────────┐
│     name     │
│   varchar    │
├──────────────┤
│ campaigns    │
│ customers    │
│ events       │
│ products     │
│ transactions │
└──────────────┘

In [9]:
con.sql("SELECT * FROM __ducklake_metadata_dataLakehouse.testing__dev.full_model;")

BinderException: Binder Error: Catalog "catalog" does not exist!

In [ ]:
con.sql("SELECT * FROM raw.products;")

CatalogException: Catalog Error: Table with name products does not exist!
Did you mean "pg_catalog.pg_proc"?

In [ ]:
con.sql("SELECT * FROM raw.transactions;")

┌────────────────┬─────────────────────┬─────────────┬────────────┬──────────┬──────────────────┬───────────────┬─────────────┬─────────────┐
│ transaction_id │      timestamp      │ customer_id │ product_id │ quantity │ discount_applied │ gross_revenue │ campaign_id │ refund_flag │
│     int64      │      timestamp      │    int64    │   double   │  int64   │      double      │    double     │    int64    │    int64    │
├────────────────┼─────────────────────┼─────────────┼────────────┼──────────┼──────────────────┼───────────────┼─────────────┼─────────────┤
│              1 │ 2021-12-27 08:25:15 │       59540 │     1630.0 │        3 │              0.0 │         43.74 │           0 │           0 │
│              2 │ 2023-06-06 21:14:26 │       54871 │     1901.0 │        3 │              0.0 │        174.78 │          21 │           0 │
│              3 │ 2023-08-31 05:29:54 │       51818 │     1884.0 │        1 │              0.0 │         40.61 │          37 │           0 │
│     

In [1]:
con.close()

NameError: name 'con' is not defined